# Лаба №3 Козлов, Ярикова. А-03 moment
## Классификация текстовых данных

| Вариант | Класс     |
|:---------:|:-----------:|
| 9       | 6, 17, 19 |

| № класса | Название класса          |
|:----------:|:--------------------------:|
| 6        | 'comp.windows.x'  |
| 17       | 'talk.politics.guns' |
| 19       | 'talk.politics.misc' |

## Код для быстрой записи данных в таблицы (к лабе не относится)

In [1]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

class Table():
    def __init__(self, title=None, data=None, headlines=["Precision", "Recall", "F1-score", "Accuracy"],
                 indexes=["Стемминг отсутствует", "Стемминг есть",
                          "Стоп-слова есть", "Стоп-слова отсечены",
                       "Взвешивание Count", "Взвешивание TF", "Взвешивание TF-IDF",
                       "max_features 100", "max_features 1000", "max_features 2000", "max_features 5000", "max_features 10000"],
                weighted=True):
        
        self.headlines = headlines
        self.dataFrame = pd.DataFrame(columns=headlines, index=indexes)
        if weighted:
            self.dataFrame.columns = pd.MultiIndex.from_tuples(
                tuple(zip(('Weighted Avg', 'Weighted Avg', 'Weighted Avg', 'Weighted Avg'), self.dataFrame.columns)))
        
    def fill_row(self, row, data=[], pred='', target='', roc_auc=None):
        """
        Заполняет строки исходя из дефолта
        row - row index
        pred='' и target='' - полученные классификатором значения и целевые значения
        Заполняется значениями Weighted Avg"""
        data = []
        data.append(round(precision_score(target, pred, average='weighted'), 3))
        data.append(round(recall_score(target, pred, average='weighted'), 3))
        data.append(round(f1_score(target, pred, average='weighted'), 3))
        data.append(round(accuracy_score(target, pred,), 3))
        for i in range(len(self.headlines)):
            self.dataFrame.loc[row][i] = data[i]   
        return self.dataFrame.loc[row]
            
    def table(self):
        return self.dataFrame
        
    def __repr__(self):
        return f'{self.dataFrame}'

delimiter = "+++++++++++++++++++++++++++++++++++++++++++"
result_table = Table(indexes=["Случайный лес с оптимальными параметрами", "Метод опорных векторов с оптимальными параметрами",
                             "Случайный лес с оптимальными параметрами(Glove)", "Метод опорных векторов с оптимальными параметрами(Glove)"])

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# П1. Загрузить выборки по варианту из лабораторной работы №2.
### Стемминг проводить не нужно.

In [3]:
from sklearn.datasets import fetch_20newsgroups
categories = ['comp.windows.x', 'talk.politics.guns', 'talk.politics.misc'] 
remove = ('headers', 'footers', 'quotes')
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42, categories=categories, remove=remove)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42, categories=categories, remove=remove)

# П2. Настройка параметров с использованием grid search

| Вариант | Метод     | Word embedding     |
|:---------:|:-----------:|:-----------:|
| 9       | RF, SVM | glove-wiki-gigaword-200 |

Случайный лес (RF):
* количество деревьев решений,
* критерий (параметр criterion: ‘gini’, ‘entropy’),
* глубина дерева (5, 15, 50, 100).

In [4]:
# Задаем параметры, которые необходимо перебрать
parameters = {'clf__criterion': ('gini', 'entropy'),
              'clf__max_depth': (5, 15, 50, 100),
              'clf__n_estimators': (10, 50, 100, 200)} 
text_clf = Pipeline([('vect', CountVectorizer(max_features=1000, stop_words='english')),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', RandomForestClassifier(criterion='gini')),])

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=3)
gs_clf.fit(twenty_train.data, twenty_train.target)
print(gs_clf.best_params_)

pred = gs_clf.predict(twenty_test.data)
print(delimiter, result_table.fill_row(row='Случайный лес с оптимальными параметрами', pred=pred, target=twenty_test.target), delimiter+'\n', sep='\n')

{'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__n_estimators': 100}
+++++++++++++++++++++++++++++++++++++++++++
Weighted Avg  Precision    0.738
              Recall       0.726
              F1-score      0.73
              Accuracy     0.726
Name: Случайный лес с оптимальными параметрами, dtype: object
+++++++++++++++++++++++++++++++++++++++++++



Метод опорных векторов (SVM):
* функция потерь (параметр kernel: ‘linear’, ‘rbf’),
* регуляризация (параметр C: {0.1, 1, 5})) Обратить внимание, что разные виды регуляризации работают с разными функциями потерь

In [5]:
# Задаем параметры, которые необходимо перебрать
parameters = {'clf__kernel': ('linear', 'rbf'),
              'clf__C': (0.1, 1, 5)}
text_clf = Pipeline([('vect', CountVectorizer(max_features=1000, stop_words='english')),
                    ('tfidf', TfidfTransformer(use_idf=True)),
                    ('clf', SVC()),])

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=3)
gs_clf.fit(twenty_train.data, twenty_train.target)
print(gs_clf.best_params_)

pred = gs_clf.predict(twenty_test.data)
print(delimiter, result_table.fill_row(row='Метод опорных векторов с оптимальными параметрами', pred=pred, target=twenty_test.target), delimiter+'\n', sep='\n')

{'clf__C': 1, 'clf__kernel': 'rbf'}
+++++++++++++++++++++++++++++++++++++++++++
Weighted Avg  Precision     0.78
              Recall       0.776
              F1-score     0.777
              Accuracy     0.776
Name: Метод опорных векторов с оптимальными параметрами, dtype: object
+++++++++++++++++++++++++++++++++++++++++++



In [6]:
SVC().get_params().keys()

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

# П3. Перевести выборку к векторному представлению word embedding согласно варианту.

| Вариант | Метод     | Word embedding     |
|:---------:|:-----------:|:-----------:|
| 9       | RF, SVM | glove-wiki-gigaword-200 |

In [7]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [8]:
# glove_model = gensim.downloader.load("glove-wiki-gigaword-200")  # load glove vector

In [9]:
# glove_model.save("glove-wiki-gigaword-200_save")
glove_model = gensim.models.KeyedVectors.load("glove-wiki-gigaword-200_save")

In [10]:
def text2vec(text_data):
    
    vectorizer = CountVectorizer(stop_words='english')
    
    # Векторизуем с помощью обученного CountVectorizer
    X = vectorizer.fit_transform(text_data)
    CV_text_data=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
#     CV_text_data
    # Создадим датафрейм, в который будем сохранять вектор документа
    glove_data=pd.DataFrame()
    words_vocab=CV_text_data.columns

    # Пробегаем по каждой строке (по каждому документу)
    for i in range(CV_text_data.shape[0]):

        # Вектор одного документа с размерностью glove-модели:
        one_doc = np.zeros(200) 

        # Пробегаемся по каждому документу, смотрим, какие слова документа присутствуют в нашем словаре
        # Суммируем glove-вектора каждого известного слова в one_doc
        for word in words_vocab[CV_text_data.iloc[i,:] >= 1]:
            if word in glove_model.key_to_index.keys(): 
                #print(word, ': ', glove_model[word])
                one_doc += glove_model[word]
        #print(text_data[i], ': ', one_doc)
        glove_data = pd.concat([glove_data, pd.DataFrame([one_doc])], axis = 0)
    #print('glove_data: ', glove_data)
    return glove_data

In [11]:
glove_train = text2vec(twenty_train.data)
glove_test = text2vec(twenty_test.data)

# П4. Провести обучение и настройку тех же алгоритмов классификации и с теми же параметрами, что и в п.2, но на векторизованной выборке

In [12]:
# Задаем параметры, которые необходимо перебрать
parameters = {'clf__criterion': ('gini', 'entropy'),
              'clf__max_depth': (5, 15, 50, 100),
              'clf__n_estimators': (10, 50, 100, 200)} 
text_clf = Pipeline([('clf', RandomForestClassifier(criterion='gini')),])

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=3)
gs_clf.fit(glove_train, twenty_train.target)
print(gs_clf.best_params_)

pred = gs_clf.predict(glove_test)
print(delimiter, result_table.fill_row(row='Случайный лес с оптимальными параметрами(Glove)', pred=pred, target=twenty_test.target), delimiter+'\n', sep='\n')

{'clf__criterion': 'entropy', 'clf__max_depth': 100, 'clf__n_estimators': 200}
+++++++++++++++++++++++++++++++++++++++++++
Weighted Avg  Precision    0.753
              Recall       0.757
              F1-score     0.752
              Accuracy     0.757
Name: Случайный лес с оптимальными параметрами(Glove), dtype: object
+++++++++++++++++++++++++++++++++++++++++++



In [13]:
# Задаем параметры, которые необходимо перебрать
parameters = {'clf__kernel': ('linear', 'rbf'),
              'clf__C': (0.1, 1, 5)}
text_clf = Pipeline([('clf', SVC()),])

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1, cv=3)
gs_clf.fit(glove_train, twenty_train.target)
print(gs_clf.best_params_)

pred = gs_clf.predict(glove_test)
print(delimiter, result_table.fill_row(row='Метод опорных векторов с оптимальными параметрами(Glove)', pred=pred, target=twenty_test.target), delimiter+'\n', sep='\n')

{'clf__C': 0.1, 'clf__kernel': 'linear'}
+++++++++++++++++++++++++++++++++++++++++++
Weighted Avg  Precision     0.74
              Recall       0.744
              F1-score      0.74
              Accuracy     0.744
Name: Метод опорных векторов с оптимальными параметрами(Glove), dtype: object
+++++++++++++++++++++++++++++++++++++++++++



# Итоговая таблица и выводы

In [14]:
result_table.table()

Weighted Avg         \
                                                      Precision Recall   
Случайный лес с оптимальными параметрами                  0.738  0.726   
Метод опорных векторов с оптимальными параметрами          0.78  0.776   
Случайный лес с оптимальными параметрами(Glove)           0.753  0.757   
Метод опорных векторов с оптимальными параметра...         0.74  0.744   

                                                                      
                                                   F1-score Accuracy  
Случайный лес с оптимальными параметрами               0.73    0.726  
Метод опорных векторов с оптимальными параметрами     0.777    0.776  
Случайный лес с оптимальными параметрами(Glove)       0.752    0.757  
Метод опорных векторов с оптимальными параметра...     0.74    0.744

С использованием __GridSearchCV__ удалось настроить оптимальные параметры
В первом случае, при использовании __TF-IDF__ лучше справился **метод опорных векторов**, однако при использовании __Glove__, обученным на википедии, лучшие результаты показал __случайный лес__. В целом, все результаты классификации получилсь неплохие.

Можно заметить ухудшение качества классификации при переходе на __Glove__, это может быть связано как с тем, что метод использует всего 200 слов, так и с тем, что он обучен на википедии, а наша выборка состоит из новостных текстов.